<a href="https://colab.research.google.com/github/hamed-jamali-software/mp32face/blob/main/coqui_XTTS_v2_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content
!git clone -b dev https://github.com/camenduru/xtts2-hf
%cd /content/xtts2-hf
!pip install -q gradio==3.50.2 TTS==0.21.1 langid unidic-lite unidic deepspeed
!pip install -q numpy<2.0.0 -U
!wget https://huggingface.co/spaces/coqui/xtts/resolve/main/examples/female.wav -O /content/xtts2-hf/examples/female.wav
!wget https://huggingface.co/spaces/coqui/xtts/resolve/main/examples/male.wav -O /content/xtts2-hf/examples/male.wav
!wget https://huggingface.co/spaces/coqui/xtts/resolve/main/ffmpeg.zip -O /content/xtts2-hf/ffmpeg.zip


/content
Cloning into 'xtts2-hf'...
remote: Enumerating objects: 518, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 518 (delta 2), reused 3 (delta 1), pack-reused 512 (from 1)
Receiving objects: 100% (518/518), 716.12 KiB | 1.95 MiB/s, done.
Resolving deltas: 100% (311/311), done.
Error downloading object: examples/female.wav (89a4fa9): Smudge error: Error downloading examples/female.wav (89a4fa9a16b6463f852cf9424f72c3d3c87aa83010e89db534c53fcd1ae12c02): [89a4fa9a16b6463f852cf9424f72c3d3c87aa83010e89db534c53fcd1ae12c02] Object does not exist on the server: [404] Object does not exist on the server

Errors logged to /content/xtts2-hf/.git/lfs/logs/20240902T175137.881776329.log
Use `git lfs logs last` to view the log.
error: external filter 'git-lfs filter-process' failed
fatal: examples/female.wav: smudge filter lfs failed
You can inspect what was checked out with 'git status'
and retry with 'git restore --source=HEAD :/'


In [2]:
import sys
import io, os, stat
import subprocess
import random
from zipfile import ZipFile
import uuid
import time
import torch
import torchaudio
import subprocess
import uuid
import time
import torch
import torchaudio
import langid
import re
from zipfile import ZipFile
from TTS.api import TTS
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts
from TTS.utils.generic_utils import get_user_data_dir
from huggingface_hub import HfApi


In [3]:
os.system('python -m unidic download')

# By using XTTS you agree to CPML license https://coqui.ai/cpml
os.environ["COQUI_TOS_AGREED"] = "1"

HF_TOKEN = os.environ.get("HF_TOKEN")
api = HfApi(token=HF_TOKEN)
repo_id = "coqui/xtts"

# Use newer ffmpeg binary for Ubuntu20 to use denoising for microphone input
print("Export newer ffmpeg binary for denoise filter")
ZipFile("ffmpeg.zip").extractall()
print("Make ffmpeg binary executable")
st = os.stat("ffmpeg")
os.chmod("ffmpeg", st.st_mode | stat.S_IEXEC)

# This will trigger downloading model
print("Downloading if not downloaded Coqui XTTS V2")
from TTS.utils.manage import ModelManager

model_name = "tts_models/multilingual/multi-dataset/xtts_v2"
ModelManager().download_model(model_name)
model_path = os.path.join(get_user_data_dir("tts"), model_name.replace("/", "--"))
print("XTTS downloaded")

Export newer ffmpeg binary for denoise filter
Make ffmpeg binary executable
 > Downloading model to /root/.local/share/tts/tts_models--multilingual--multi-dataset--xtts_v2
 > Model's license - CPML
 > Check https://coqui.ai/cpml.txt for more info.
XTTS downloaded


In [4]:
config = XttsConfig()
config.load_json(os.path.join(model_path, "config.json"))

model = Xtts.init_from_config(config)
model.load_checkpoint(
    config,
    checkpoint_path=os.path.join(model_path, "model.pth"),
    vocab_path=os.path.join(model_path, "vocab.json"),
    eval=True,
    use_deepspeed=True,
)
model.cuda()

DEVICE_ASSERT_DETECTED = 0
DEVICE_ASSERT_PROMPT = None
DEVICE_ASSERT_LANG = None

supported_languages = config.languages


/usr/local/lib/python3.10/dist-packages/TTS/utils/io.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=map_location, **kwargs)


[2024-09-02 17:55:37,643] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-09-02 17:55:39,794] [INFO] [logging.py:96:log_dist] [Rank -1] DeepSpeed info: version=0.15.0, git-hash=unknown, git-branch=unknown
[2024-09-02 17:55:39,796] [WARNING] [config_utils.py:70:_process_deprecated_field] Config parameter replace_method is deprecated. This parameter is no longer needed, please remove from your call to DeepSpeed-inference
[2024-09-02 17:55:39,799] [WARNING] [config_utils.py:70:_process_deprecated_field] Config parameter mp_size is deprecated use tensor_parallel.tp_size instead
[2024-09-02 17:55:39,801] [INFO] [logging.py:96:log_dist] [Rank -1] quantize_bits = 8 mlp_extra_grouping = False, quantize_groups = 1


Using /root/.cache/torch_extensions/py310_cu121 as PyTorch extensions root...
Creating extension directory /root/.cache/torch_extensions/py310_cu121/transformer_inference...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py310_cu121/transformer_inference/build.ninja...
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:1965: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Building extension module transformer_inference...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


Time to load transformer_inference op: 82.68424773216248 seconds
[2024-09-02 17:57:03,228] [INFO] [logging.py:96:log_dist] [Rank -1] DeepSpeed-Inference config: {'layer_id': 0, 'hidden_size': 1024, 'intermediate_size': 4096, 'heads': 16, 'num_hidden_layers': -1, 'dtype': torch.float32, 'pre_layer_norm': True, 'norm_type': <NormType.LayerNorm: 1>, 'local_rank': -1, 'stochastic_mode': False, 'epsilon': 1e-05, 'mp_size': 1, 'scale_attention': True, 'triangular_masking': True, 'local_attention': False, 'window_size': 1, 'rotary_dim': -1, 'rotate_half': False, 'rotate_every_two': True, 'return_tuple': True, 'mlp_after_attn': True, 'mlp_act_func_type': <ActivationFuncType.GELU: 1>, 'specialized_mode': False, 'training_mp_size': 1, 'bigscience_bloom': False, 'max_out_tokens': 1024, 'min_out_tokens': 1, 'scale_attn_by_inverse_layer_idx': False, 'enable_qkv_quantization': False, 'use_mup': False, 'return_single_tuple': False, 'set_empty_params': False, 'transposed_mode': False, 'use_triton': Fa

Loading extension module transformer_inference...


In [25]:
import textwrap
import subprocess
import uuid
import time
import re
import datetime
import csv
from io import StringIO
from google.colab import files

In [6]:
def split_text(text, max_length=133):
    return textwrap.wrap(text, width=max_length, break_long_words=False)


In [13]:
import numpy as np

def create_silence(duration_seconds, file_path, sample_rate=24000):
    # تعداد نمونه‌ها به ازای مدت زمان
    num_samples = int(duration_seconds * sample_rate)
    # ایجاد آرایه‌ای از صفرها (صداهای خالی)
    silence = np.zeros(num_samples, dtype=np.float32)
    # ذخیره فایل WAV
    torchaudio.save(file_path, torch.tensor([silence]), sample_rate)

In [23]:
def text_to_speech(prompt, language, audio_file_pth=None, mic_file_path=None, use_mic=False, voice_cleanup=False, no_lang_auto_detect=False, agree=True, output_file=None):
    if agree:


        speaker_wav = audio_file_pth




        try:
            prompt = re.sub("([^\x00-\x7F]|\w)(\.|\。|\?)", r"\1 \2\2", prompt)
            print("Generating new audio...")
            # t0 = time.time() # زمان شروع فرآیند تولید صوت
            out = model.inference(
                prompt,
                language,
                *model.get_conditioning_latents(audio_path=speaker_wav, gpt_cond_len=30, gpt_cond_chunk_len=4, max_ref_length=60),
                repetition_penalty=5.0,
                temperature=0.75,
            )
            # inference_time = time.time() - t0
            # print(f"Time to generate audio: {round(inference_time * 1000)} milliseconds")
            # real_time_factor = (time.time() - t0) / out['wav'].shape[-1] * 24000
            # print(f"Real-time factor (RTF): {real_time_factor}")
            torchaudio.save(output_file, torch.tensor(out["wav"]).unsqueeze(0), 24000)
            print("Audio saved to  "+output_file + "  sentence : " + prompt)

        except RuntimeError as e:
            if "device-side assert" in str(e):
                print(f"Exit due to: Unrecoverable exception caused by language:{language} prompt:{prompt}")
                print("Unhandled Exception encounter, please retry in a minute")
                print("Cuda device-assert Runtime encountered need restart")
                if not DEVICE_ASSERT_DETECTED:
                    DEVICE_ASSERT_DETECTED = 1
                    DEVICE_ASSERT_PROMPT = prompt
                    DEVICE_ASSERT_LANG = language

                error_time = datetime.datetime.now().strftime("%d-%m-%Y-%H:%M:%S")
                error_data = [
                    error_time,
                    prompt,
                    language,
                    audio_file_pth,
                    mic_file_path,
                    use_mic,
                    voice_cleanup,
                    no_lang_auto_detect,
                    agree,
                ]
                error_data = [str(e) if type(e) != str else e for e in error_data]
                print(error_data)
                write_io = StringIO()
                csv.writer(write_io).writerows([error_data])
                csv_upload = write_io.getvalue().encode()

                filename = error_time + "_" + str(uuid.uuid4()) + ".csv"
                print("Writing error csv")
                error_api = HfApi()
                error_api.upload_file(
                    path_or_fileobj=csv_upload,
                    path_in_repo=filename,
                    repo_id="coqui/xtts-flagged-dataset",
                    repo_type="dataset",
                )

                print("Writing error reference audio")
                speaker_filename = error_time + "_reference_" + str(uuid.uuid4()) + ".wav"
                error_api.upload_file(
                    path_or_fileobj=speaker_wav,
                    path_in_repo=speaker_filename,
                    repo_id="coqui/xtts-flagged-dataset",
                    repo_type="dataset",
                )

                space = api.get_space_runtime(repo_id=repo_id)
                if space.stage != "BUILDING":
                    api.restart_space(repo_id=repo_id)
                else:
                    print("TRIED TO RESTART but space is building")
            else:
                if "Failed to decode" in str(e):
                    print("Speaker encoding error", str(e))
                else:
                    print("RuntimeError: non device-side assert error:", str(e))
                return None

        return output_file


In [8]:
def generate_speech(prompt, language_code, chunk_id):
    output_file = f"chunk_{chunk_id}.wav"
    language_code = "en"
    audio_file_path = '/content/5.mp3'  # Path to reference audio file if needed
    mic_file_path = None  # Path to microphone recorded audio if needed
    use_microphone = False
    voice_cleanup = True
    disable_lang_auto_detect = True


    speaker_wav = audio_file_path

    # Filtering for microphone input
    lowpassfilter = denoise = trim = loudness = True
    lowpass_highpass = "lowpass=8000,highpass=75," if lowpassfilter else ""
    trim_silence = "areverse,silenceremove=start_periods=1:start_silence=0:start_threshold=0.02,areverse,silenceremove=start_periods=1:start_silence=0:start_threshold=0.02," if trim else ""

    out_filename = speaker_wav + "filter.wav"
    shell_command = f"./ffmpeg -y -i {speaker_wav} -af {lowpass_highpass}{trim_silence} {out_filename}".split(" ")
    subprocess.run(shell_command, capture_output=False, text=True, check=True)
    speaker_wav = out_filename

    return text_to_speech(
        prompt=prompt,
        language=language_code,
        audio_file_pth=speaker_wav,
        mic_file_path=mic_file_path,
        use_mic=use_microphone,
        voice_cleanup=voice_cleanup,
        no_lang_auto_detect=disable_lang_auto_detect,
        agree=True,
        output_file= output_file)

In [9]:
# Function to combine multiple audio files into one
def combine_audio_files(audio_files, output_file):

    with open("file_list.txt", "w") as file_list:
      for audio_file in audio_files:
          file_list.write(f"file '{audio_file}'\n")

    command = ["ffmpeg", "-f", "concat", "-safe", "0", "-i", "file_list.txt", "-c", "copy", output_file]

    try:
        result = subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        print(f"Audio files successfully combined into {output_file}")
        print(result.stdout)
        print(result.stderr)
    except subprocess.CalledProcessError as e:
        print(f"Error occurred during combining audio files: {e}")
        print(e.stdout)
        print(e.stderr)

    for audio_file in audio_files:
      os.remove(audio_file)

    # with open(output_file, 'wb') as outfile:
    #     for audio_file in audio_files:
    #         with open(audio_file, 'rb') as infile:
    #             outfile.write(infile.read())
            # Clean up temporary files
            # os.remove(audio_file)

In [26]:
def process_text_to_speech(text, language_code):
    # Split text into chunks of 200 characters
    chunks = split_text(text)

    sentences = re.split(r'(?<=[.!?]|\[|\])\s+', text)

    all_audio_files = []


    if os.path.isfile("combined_output.wav"):
      os.remove("combined_output.wav")

    total_sentences = len(sentences)
    for idx, sentence in enumerate(sentences):
        print(f"Processing sentence {idx + 1} of {total_sentences}...")
        temp_file = generate_speech(sentence, language_code, idx + 1)
        if os.path.exists(temp_file):
            all_audio_files.append(temp_file)

            # ایجاد فایل صوتی خالی با مدت زمان رندوم
            silence_duration = random.uniform(1, 2)  # مدت زمان رندوم بین 1 تا 2 ثانیه
            silence_file = f"silence_{idx + 1}.wav"
            create_silence(silence_duration, silence_file)
            all_audio_files.append(silence_file)
        else:
            print(f"Failed to generate audio for sentence {idx + 1}.")

    # Combine all audio files into one
    if all_audio_files:
        combined_audio = "combined_output.wav"
        combine_audio_files(all_audio_files, combined_audio)
        print(f"All audio files combined into {combined_audio}")

        files.download("combined_output.wav")

    else:
        print("No audio files to combine.")

In [27]:

# Example usage
prompt_text = """

Hello

I’m hamjam, and in today’s video, we’re going to explore [When I realized this, I changed my life in 90 days].

I can't emphasize enough how powerful this video can be in shaping your journey. In just a few minutes, you'll understand why. But first, let me ask you a question: How many weeks do you think the average person lives? Don’t overthink it—just say the first number that comes to mind. I guessed 96,000 weeks. My brother said 100,000 weeks. Some friends guessed 150,000 or even 200,000 weeks. But do you know what's interesting? The average person only lives around 4,000 weeks.

It’s surprising, isn’t it? When you think about life in terms of weeks, the number seems so small. And when you consider that you might have already spent 1,000 or more of those weeks, it becomes clear how precious time really is. In this video, we’re going to decide what to do with the weeks you have left. Right now, your brain might be telling you to skip this, to watch something more fun instead. But let me tell you this: in a year, two years, or even five years from now, you might wish you had watched this all the way through and made a decision.

You see, most people live robotic lives. They wake up tired because they stayed up late scrolling through their phones. They go to a job they don’t like, come home feeling drained, and distract themselves with more screens. Day by day, their remaining weeks slip away, unnoticed, until they retire—looking back at their lives, wondering where the time went.

Don’t let that be you. Don’t wait until you have only a few hundred weeks left to live and wonder why you didn’t make better use of your time. The truth is, most people never think about death; they live as if they’ll live forever, stuck in routines that offer no real satisfaction. They’re zombies, going through the motions without ever really being present in their own lives.

But you have a choice. You can decide, right now, how you want to spend the weeks you have left. You can choose to live with intention, to pursue your passions, to be fully present in each moment. Or, you can do nothing and end up like the rest—living a life full of regrets, looking back and wishing you had done things differently.

This video isn’t meant to scare you; it’s meant to empower you. It’s a reminder that you don’t have to live like everyone else. You don’t have to be stuck in a cycle of endless distraction, giving in to every impulse for instant gratification. You can choose to break free, to focus, to build a life you’re proud of.

But that choice is yours to make. I can’t make it for you. I can only guide you to the edge of this realization—you have to take the leap. So, as you watch this video, really think about how you want to live. Are you going to be the best version of yourself, or are you going to settle for less?

Remember, the pain of discipline weighs ounces, but the pain of regret weighs tons. The weeks are ticking by—how are you going to spend them?

I hope you found today’s video helpful and inspiring. If you enjoyed it, please give us a thumbs up and share it with anyone who might benefit from it.

Remember to subscribe if you haven’t already, so you never miss out on our latest tips and insights. Feel free to leave a comment below with your thoughts or any questions you have — I love hearing from you!


"""
process_text_to_speech(prompt_text, language_code="en")

Processing sentence 1 of 44...
Generating new audio...
Audio saved to  chunk_1.wav  sentence : 

Hello 

I’m hamjam, and in today’s video, we’re going to explore [When I realized this, I changed my life in 90 days].
Processing sentence 2 of 44...
Generating new audio...
Audio saved to  chunk_2.wav  sentence : I can't emphasize enough how powerful this video can be in shaping your journey ..
Processing sentence 3 of 44...
Generating new audio...
Audio saved to  chunk_3.wav  sentence : In just a few minutes, you'll understand why ..
Processing sentence 4 of 44...
Generating new audio...
Audio saved to  chunk_4.wav  sentence : But first, let me ask you a question: How many weeks do you think the average person lives ??
Processing sentence 5 of 44...
Generating new audio...
Audio saved to  chunk_5.wav  sentence : Don’t overthink it—just say the first number that comes to mind ..
Processing sentence 6 of 44...
Generating new audio...
Audio saved to  chunk_6.wav  sentence : I guessed 96,000 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
command = [
    "ffmpeg",
    "-y",
    "-i", "combined_output.wav",
    "-filter:a", "atempo=0.93",
    "output_final.wav"
]


# اجرای فرمان FFmpeg
try:
    result = subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    print("Command executed successfully.")




    print(result.stdout)
except subprocess.CalledProcessError as e:
    print("Error occurred:")
    print(e.stderr)

Command executed successfully.

